# Assignment 2: Bias in Data
## Emily Linebarger

### 1. Data extraction

The two datasets I'll be using for this analysis are the Politicians by Country dataset from FigShare (https://figshare.com/articles/dataset/Untitled_Item/5513449) and the World Population Data Sheet (https://docs.google.com/spreadsheets/d/1CFJO2zna2No5KqNm9rPK5PCACoXKzb-nycJFhV689Iw/edit#gid=283125346), from the Population Reference Bureau (https://www.prb.org/international/indicator/population/table/).

All data was downloaded on October 9, 2021 and was placed in the "raw" folder without edits. 

### 2. Data cleaning

In [94]:
import pandas as pd
import numpy as np

# First, clean the data on politicians by country
politicians = pd.read_csv("raw/country/data/page_data.csv")

In [95]:
politicians.head()

,page,country,rev_id
0,Template:ZambiaProvincialMinisters,Zambia,235107991
1,Bir I of Kanem,Chad,355319463
2,Template:Zimbabwe-politician-stub,Zimbabwe,391862046
3,Template:Uganda-politician-stub,Uganda,391862070
4,Template:Namibia-politician-stub,Namibia,391862409


In [96]:
politicians.shape

(47197, 3)

In [97]:
# All of the 'page' rows that start with "Template" are not Wikipedia articles, and should be dropped. 
mask = politicians.page.str.contains("^Template")
politicians[mask]

,page,country,rev_id
0,Template:ZambiaProvincialMinisters,Zambia,235107991
2,Template:Zimbabwe-politician-stub,Zimbabwe,391862046
3,Template:Uganda-politician-stub,Uganda,391862070
4,Template:Namibia-politician-stub,Namibia,391862409
5,Template:Nigeria-politician-stub,Nigeria,391862819
...,...,...,...
44916,Template:New Zealand prime minister electoral ...,New Zealand,806286945
44966,Template:Current New Zealand political party l...,New Zealand,806301302
45587,Template:Lists of US Presidents and Vice Presi...,United States,806668141
45823,Template:Prime Ministers of Australia,Australia,806799996


In [98]:
politicians = politicians[~mask]
politicians.shape # This drops 496 rows. 
politicians.to_csv('clean/politicians.csv')

In [99]:
# Next, clean the population data. 
# There are some regional aggregates, which are distinguished by all-caps in the 'geography' field.
# These won't match the country strings in the politicians dataset, but they're important to keep around 
# to get regional aggregates. 
population = pd.read_csv('raw/WPDS_2020_data - WPDS_2020_data.csv.csv')
population = population.rename(columns={'Name':'country'}) # Rename to match politicians schema
population.head()

,FIPS,country,Type,TimeFrame,Data (M),Population
0,WORLD,WORLD,World,2019,7772.850,7772850000
1,AFRICA,AFRICA,Sub-Region,2019,1337.918,1337918000
2,NORTHERN AFRICA,NORTHERN AFRICA,Sub-Region,2019,244.344,244344000
3,DZ,Algeria,Country,2019,44.357,44357000
4,EG,Egypt,Country,2019,100.803,100803000


In [100]:
population.to_csv('clean/population.csv')

### 3. Getting article quality predictions

To get article quality scores, I will use the ORES API, which uses a machine-learning model to attach a quality score to a given revision ID. 
Documentation is here: https://ores.wikimedia.org/v3/#!/scoring/get_v3_scores_context_revid_model

For each group of revision IDs, I'll need to build up a URL string of the format: 
https://ores.wikimedia.org/v3/scores/enwiki?models=articlequality&revids=355319463%7C498683267
This queries the "enwiki" database (the content parameter), with the "articlequality" model (model parameter). 
From the API documentation, the database errors when more than 200 revision IDs are queried, so I'll query them in batches and write out temporary files. 

In [101]:
import requests
import json
from datetime import datetime
import os

def query_api_batch(start_idx, end_idx, data, date):
    # Get the revision IDs from the start to the end index
    rev_ids = data.rev_id[start_idx:end_idx].astype('int')
    rev_ids = rev_ids.astype('str')
    rev_ids = '|'.join(rev_ids.to_list())
    
    # Create a datetime string for data saving
    date = datetime.today().strftime("%Y_%m_%d_%H_%M_%S")
    os.makedirs(f"api_queries_raw/{date}", exist_ok = True)
    os.makedirs(f"cleaned_queries/{date}", exist_ok = True)
    
    # Query the API
    r = requests.get(f"https://ores.wikimedia.org/v3/scores/enwiki?models=articlequality&revids={rev_ids}")
    
    # Manipulate the data to get the 'prediction' column for each ID
    data = json.loads(r.text)
    # Save this query output
    with open(f'api_queries_raw/{date}/{start_idx}_{end_idx}.txt', 'w') as outfile:
        json.dump(data, outfile)
        
    # Extract just the columns you need from the queries - prediction and revision ID
    cleaned_data = dict()
    for rev_id in data['enwiki']['scores'].keys():
        if 'error' in data['enwiki']['scores'][rev_id]['articlequality'].keys():
            score = np.nan
        else:
            score = data['enwiki']['scores'][rev_id]['articlequality']['score']['prediction']
        cleaned_data[rev_id] = score
    cleaned_data = pd.DataFrame({'rev_id': cleaned_data.keys(), 'score': cleaned_data.values()})
    cleaned_data.to_csv(f'cleaned_queries/{date}/{start_idx}_{end_idx}.csv')

In [102]:
# First, read in past results. The API starts to reject requests after a certain number of queries, so I had
# to query in batches and save results to disk. 
# ** Note - for the first two runs on 10/9/2021 and 10/11/2021, I did not save the time. So I've given these 
# folders a time of midnight (00_00_00).
from pathlib import Path
all_dates = [x for x in Path('cleaned_queries').iterdir()]
previous_results = list()
for date in all_dates:
    previous_results.extend([x for x in date.iterdir() if x.is_file()])
print(f"Previous results found: {len(previous_results)}")

Previous results found: 1007


In [103]:
# Glob all of these results together 
wiki_codes = []

for filename in previous_results:
    df = pd.read_csv(filename)
    wiki_codes.append(df)

wiki_codes = pd.concat(wiki_codes, axis=0, ignore_index=True)

In [104]:
wiki_codes.head()

,Unnamed: 0,rev_id,score
0,0,699260156,NaN
1,1,708813010,NaN
2,2,715457941,NaN
3,3,717369009,Stub
4,4,717927381,NaN


In [105]:
wiki_codes.shape

(47783, 3)

In [106]:
# Merge these results onto data, so you only query lines that are missing 
data = pd.read_csv('clean/politicians.csv')
data['rev_id'] = np.round(data['rev_id'])
data.head()

,Unnamed: 0,page,country,rev_id
0,1,Bir I of Kanem,Chad,355319463
1,10,Information Minister of the Palestinian Nation...,Palestinian Territory,393276188
2,12,Yos Por,Cambodia,393822005
3,23,Julius Gregr,Czech Republic,395521877
4,24,Edvard Gregr,Czech Republic,395526568


In [107]:
wiki_codes = wiki_codes[['rev_id', 'score']]
scored_data = data.merge(wiki_codes, on = 'rev_id', how = 'outer')
scored_data.head()

,Unnamed: 0,page,country,rev_id,score
0,1,Bir I of Kanem,Chad,355319463,Stub
1,10,Information Minister of the Palestinian Nation...,Palestinian Territory,393276188,Stub
2,12,Yos Por,Cambodia,393822005,Stub
3,23,Julius Gregr,Czech Republic,395521877,Stub
4,24,Edvard Gregr,Czech Republic,395526568,Stub


In [108]:
# Save this data out
has_scores = scored_data.loc[~scored_data.score.isnull()]
has_scores.to_csv('clean/pages_with_scores.csv')

In [109]:
# Pull out the missing lines, and query the database for their scores. 
missing_scores = scored_data.loc[scored_data.score.isnull()]
missing_scores = missing_scores.drop_duplicates()
missing_scores.head()

,Unnamed: 0,page,country,rev_id,score
14,126,List of politicians in Poland,Poland,516633096,NaN
25,222,Tingtingru,Vanuatu,550682925,NaN
59,330,Daud Arsala,Afghanistan,627547024,NaN
87,359,Book:Two Political Biographies,India,636911471,NaN
196,514,Dilaver Bey,Turkey,669987106,NaN


In [110]:
missing_scores.shape

(277, 5)

In [111]:
# Save the results you were unable to score to disk
missing_scores.to_csv('clean/unable_to_score_pages.csv')

In [112]:
# Iterate through the entire dataset, and save all query results
# There are 277 pages that couldn't be scored. Iterate through this loop again in you find more than this. 
if missing_scores.shape[0] > 277:
    # Create a datetime string for data saving
    date = datetime.today().strftime("%Y_%m_%d_%H_%M_%S")
    os.makedirs(f"api_queries_raw/{date}", exist_ok = True)
    os.makedirs(f"cleaned_queries/{date}", exist_ok = True)
    
    # Iterate through missing data
    step_size = 50
    for i in range(0, missing_scores.shape[0], step_size):
        start_idx = i # First start index will be 0, then 50, 100, etc.
        end_idx = i + (step_size - 1) # First end index will be 49, then 99, 149, etc. 
        if (end_idx > missing_scores.shape[0]):
            print("Reached the end!")
            end_idx = missing_scores.shape[0] # If you've reached the end, only query the remaining IDs available

        query_api_batch(start_idx, end_idx, missing_scores, date)
        print(f"Start at idx {start_idx}, end at idx {end_idx}")
else:
    print("All pages have been scored!")

All pages have been scored!


# 4. Combining the datasets

Now, I'll merge the scored pages with the population data. 

In [113]:
scored_politicians = pd.read_csv('clean/pages_with_scores.csv')
population = pd.read_csv('clean/population.csv')

# Do an outer merge on the 'country' column, so nonmatching observations are kept.
results = scored_politicians.merge(population, on='country', how='outer')
results = results[['page', 'country', 'rev_id', 'score', 'FIPS', 'Type', 'TimeFrame', 'Data (M)', 'Population']]
results.head()

,page,country,rev_id,score,FIPS,Type,TimeFrame,Data (M),Population
0,Bir I of Kanem,Chad,355319463.0,Stub,TD,Country,2019.0,16.877,16877000.0
1,Abdullah II of Kanem,Chad,498683267.0,Stub,TD,Country,2019.0,16.877,16877000.0
2,Salmama II of Kanem,Chad,565745353.0,Stub,TD,Country,2019.0,16.877,16877000.0
3,Kuri I of Kanem,Chad,565745365.0,Stub,TD,Country,2019.0,16.877,16877000.0
4,Mohammed I of Kanem,Chad,565745375.0,Stub,TD,Country,2019.0,16.877,16877000.0


In [114]:
# Write to disk any rows that did not exist in both datasets 
no_match = results.loc[(results.Population.isnull()) | (results.score.isnull())]
no_match.to_csv("clean/wp_wpds_countries-no_match.csv")

In [115]:
# Save the results that did match.
match = results.loc[~results.rev_id.isin(no_match.rev_id)]
match = match[['country', 'page', 'rev_id', 'score', 'Population']]
match.columns = ['country', 'article_name', 'revision_id', 'article_quality_est', 'population']
match.to_csv("clean/wp_wpds_politicians_by_country.csv")

# 5. Analysis

For the analysis, I will calculate the proportion of articles per population and high quality articles for each country/geographic region. I define "high quality" as having either a "FA" or "GA" score.

In [116]:
match.head()

,country,article_name,revision_id,article_quality_est,population
0,Chad,Bir I of Kanem,355319463.0,Stub,16877000.0
1,Chad,Abdullah II of Kanem,498683267.0,Stub,16877000.0
2,Chad,Salmama II of Kanem,565745353.0,Stub,16877000.0
3,Chad,Kuri I of Kanem,565745365.0,Stub,16877000.0
4,Chad,Mohammed I of Kanem,565745375.0,Stub,16877000.0


In [117]:
match.article_quality_est.unique()

array(['Stub', 'Start', 'C', 'B', 'GA', 'FA'], dtype=object)

In [118]:
# First, calculate the total number of articles by a country's population.
articles_per_population = match.copy()
articles_per_population['num_articles'] = 1 # Create a count variable to collapse by
articles_per_population = articles_per_population.groupby(['country', 'population'])['num_articles'].sum().reset_index()

# Then, generate the proportion of "number of articles" / "population" as a percentage
articles_per_population['articles_per_population'] = (articles_per_population['num_articles'] / articles_per_population['population'])*100

articles_per_population.head()

,country,population,num_articles,articles_per_population
0,Afghanistan,38928000.0,319,0.000819
1,Albania,2838000.0,456,0.016068
2,Algeria,44357000.0,116,0.000262
3,Andorra,82000.0,34,0.041463
4,Angola,32522000.0,106,0.000326


In [119]:
# Now, calculate the percentage of high quality articles. 
# Out of the total number of high-quality articles in a country, how many are high-quality? 
article_quality = match.copy()
article_quality['num_articles'] = 1
article_quality['quality_article'] = article_quality.article_quality_est.isin(['GA', 'FA']).astype('int')

# Now, sum these two columns and create the proportion column
article_quality = article_quality.groupby('country').agg({'num_articles':'sum', 'quality_article':'sum'}).reset_index()
article_quality['pct_quality_articles'] = (article_quality['quality_article'] / article_quality['num_articles'])*100

article_quality.head()

,country,num_articles,quality_article,pct_quality_articles
0,Afghanistan,319,13,4.075235
1,Albania,456,3,0.657895
2,Algeria,116,2,1.724138
3,Andorra,34,0,0.000000
4,Angola,106,0,0.000000


In [124]:
# Finally, prepare regional aggregates. First, make a map of regions to countries. 
regions_to_countries = pd.read_csv('clean/population.csv')
regions_to_countries.head()

,Unnamed: 0,FIPS,country,Type,TimeFrame,Data (M),Population
0,0,WORLD,WORLD,World,2019,7772.850,7772850000
1,1,AFRICA,AFRICA,Sub-Region,2019,1337.918,1337918000
2,2,NORTHERN AFRICA,NORTHERN AFRICA,Sub-Region,2019,244.344,244344000
3,3,DZ,Algeria,Country,2019,44.357,44357000
4,4,EG,Egypt,Country,2019,100.803,100803000


In [125]:
# Each sub-region is a header over the countries it contains. So, extend these down.
regions_to_countries['region'] = regions_to_countries['country']
regions_to_countries.loc[regions_to_countries.Type!='Sub-Region', 'region'] = np.nan
regions_to_countries['region'] = regions_to_countries['region'].fillna(method = 'ffill')

regions_to_countries = regions_to_countries[['country', 'region', 'Population']]
regions_to_countries = regions_to_countries.rename(columns={'Population':'regional_population'})
regions_to_countries.head()

,country,region,regional_population
0,WORLD,NaN,7772850000
1,AFRICA,AFRICA,1337918000
2,NORTHERN AFRICA,NORTHERN AFRICA,244344000
3,Algeria,NORTHERN AFRICA,44357000
4,Egypt,NORTHERN AFRICA,100803000


In [126]:
regions_to_countries.to_csv('clean/regions_to_countries_map.csv')

In [127]:
# Merge this regional data onto both the articles-per-population and article-quality datasets. 
# Only keep the rows from the original dataset, so aggregate region names will be dropped. 
articles_per_population = articles_per_population.merge(regions_to_countries, on = 'country', how = 'left')
articles_per_population.head()

,country,population,num_articles,articles_per_population,region_x,Population,region_y,regional_population
0,Afghanistan,38928000.0,319,0.000819,SOUTH ASIA,38928000,SOUTH ASIA,38928000
1,Albania,2838000.0,456,0.016068,SOUTHERN EUROPE,2838000,SOUTHERN EUROPE,2838000
2,Algeria,44357000.0,116,0.000262,NORTHERN AFRICA,44357000,NORTHERN AFRICA,44357000
3,Andorra,82000.0,34,0.041463,SOUTHERN EUROPE,82000,SOUTHERN EUROPE,82000
4,Angola,32522000.0,106,0.000326,MIDDLE AFRICA,32522000,MIDDLE AFRICA,32522000


In [128]:
article_quality = article_quality.merge(regions_to_countries, on = 'country', how = 'left')
article_quality.head()

,country,num_articles,quality_article,pct_quality_articles,region,regional_population
0,Afghanistan,319,13,4.075235,SOUTH ASIA,38928000
1,Albania,456,3,0.657895,SOUTHERN EUROPE,2838000
2,Algeria,116,2,1.724138,NORTHERN AFRICA,44357000
3,Andorra,34,0,0.000000,SOUTHERN EUROPE,82000
4,Angola,106,0,0.000000,MIDDLE AFRICA,32522000


# 6. Results

Table 1: 10 highest-ranked countries in terms of articles-per-population

In [66]:
articles_per_population = articles_per_population.sort_values(by='articles_per_population', ascending = False)
articles_per_population.head(10)

,country,population,num_articles,articles_per_population
169,Tuvalu,10000.0,54,0.540000
117,Nauru,11000.0,52,0.472727
138,San Marino,34000.0,81,0.238235
110,Monaco,38000.0,40,0.105263
95,Liechtenstein,39000.0,28,0.071795
104,Marshall Islands,57000.0,37,0.064912
164,Tonga,99000.0,63,0.063636
70,Iceland,368000.0,201,0.054620
3,Andorra,82000.0,34,0.041463
52,Federated States of Micronesia,106000.0,36,0.033962


Table 2: 10 lowest-ranked countries in terms of articles-per-population

In [67]:
articles_per_population = articles_per_population.sort_values(by='articles_per_population', ascending = True)
articles_per_population.head(10)

,country,population,num_articles,articles_per_population
71,India,1.400100e+09,968,0.000069
72,Indonesia,2.717390e+08,209,0.000077
34,China,1.402385e+09,1129,0.000081
176,Uzbekistan,3.417400e+07,28,0.000082
51,Ethiopia,1.149160e+08,101,0.000088
181,Zambia,1.838400e+07,25,0.000136
84,"Korea, North",2.577900e+07,36,0.000140
162,Thailand,6.653400e+07,112,0.000168
114,Mozambique,3.116600e+07,58,0.000186
13,Bangladesh,1.698090e+08,317,0.000187


Table 3: 10 highest-ranked countries in terms of quality article percentage

In [68]:
article_quality = article_quality.sort_values(by='pct_quality_articles', ascending = False)
article_quality.head(10)

,country,num_articles,quality_article,pct_quality_articles
84,"Korea, North",36,8,22.222222
140,Saudi Arabia,117,15,12.820513
135,Romania,343,42,12.244898
31,Central African Republic,66,8,12.121212
176,Uzbekistan,28,3,10.714286
106,Mauritania,48,5,10.416667
64,Guatemala,83,7,8.433735
44,Dominica,12,1,8.333333
158,Syria,128,10,7.812500
18,Benin,91,7,7.692308


Table 4: 10 lowest-ranked countries in terms of quality article percentage

In [69]:
article_quality = article_quality.sort_values(by='pct_quality_articles', ascending = True)
article_quality.head(10)

,country,num_articles,quality_article,pct_quality_articles
11,Bahamas,20,0,0.0
54,Finland,569,0,0.0
104,Marshall Islands,37,0,0.0
168,Turkmenistan,32,0,0.0
166,Tunisia,138,0,0.0
43,Djibouti,37,0,0.0
52,Federated States of Micronesia,36,0,0.0
164,Tonga,63,0,0.0
3,Andorra,34,0,0.0
5,Antigua and Barbuda,24,0,0.0


Table 5: Ranking of geographic regions (in descending order) in terms of total count of politician articles over regional population

Table 6: Ranking of geographic regions (in descending order) in terms of the relative proportion of politician articles that are of high quality (ranked FA or GA)